## Frameworks de alto nivel (PyTorch Lightning)

<a target="_blank" href="https://github.com/pglez82/DeepLearningWeb/blob/master/labs/notebooks/Frameworks%20alto%20nivel%20(PyTorch%20Lightning).ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

PyTorch Lightning es una biblioteca de código abierto diseñada para simplificar y acelerar el proceso de entrenamiento de modelos de aprendizaje profundo utilizando PyTorch. Proporciona una abstracción de alto nivel sobre PyTorch que ayuda a los investigadores y desarrolladores a escribir código más limpio y legible, al tiempo que aprovechan al máximo la flexibilidad y potencia de PyTorch.

Una de las principales ventajas de PyTorch Lightning es su enfoque en la modularidad y la organización del código. Proporciona una estructura clara y consistente para definir modelos, optimizadores, funciones de pérdida y bucles de entrenamiento y validación. Esto facilita la reutilización de código y la experimentación rápida con diferentes arquitecturas y configuraciones de entrenamiento.

Además, PyTorch Lightning se encarga de tareas como la configuración de dispositivos (CPU o GPU), la administración automática de la memoria, el seguimiento de métricas durante el entrenamiento y la gestión de la generación de registros (logs). También incluye características avanzadas, como el entrenamiento distribuido y la integración con bibliotecas populares de visualización y registro.

### Instalación

In [ ]:
!pip install lightning==2.0.1
!pip install ipympl # para gráficos interactivos

### Definición de la red
En este caso vamos a entrenar un pequeño **autoencoder** para ejemplificar el uso de PyTorch Lightning. En lugar de utilizar como clase base la clase `torch.nn.Module`, utilizaremos `pl.LightningModule`.

In [ ]:
from torch import optim, nn
import lightning.pytorch as pl
import torchmetrics as tm
import torch.nn.functional as F

# Definimos la red
class AutoEncoder(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
        self.decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))
        # Utilizado para acumular el loss en el bucle de validación
        self.validation_loss = tm.MeanSquaredError()

    # Este método define el bucle de entrenamiento
    def training_step(self, batch, batch_idx):
        #Al ser un autoencoder, no necesitamos la y
        x, _ = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, _ = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.validation_loss(x_hat, x)
        self.log("val_loss", loss, prog_bar=True)

    def on_validation_epoch_end(self):
        val_loss = self.validation_loss.compute()
        self.log("val_loss_epoch", val_loss)

    def predict_step(self, batch, batch_idx):
        x, _ = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        return z

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
model = AutoEncoder()

### Cargamos el Dataset
En este caso utilizaremos la clase que PyTorch Lightning provee para la carga de datos: `pl.LightningDataModule`. Para ello debemos de implementar una serie de métodos que devolverán los DataLoaders correspondientes:

In [ ]:
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import random_split, DataLoader

class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=32, num_workers=2):
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.transform = transforms.Compose([transforms.ToTensor()])

    def prepare_data(self):
        # descargamos los datos
        MNIST(root='data', train=True, download=True)
        MNIST(root='data', train=False, download=True)

    def setup(self, stage):
        if stage == "fit":
            mnist_full = MNIST(root='data', train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [50000, 10000])
        if stage == "predict":
            self.mnist_test = MNIST(root='data', train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size, num_workers=self.num_workers)

    def predict_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=self.num_workers)

mnist = MNISTDataModule(batch_size=128, num_workers=8)

### Entrenando el modelo
Para entrenar el modelo solo necesitaremos crear una instancia de la clase `Trainer` y llamar al método `fit`. ESta clase nos ecapsula, entre otras cosas, las siguientes características:


- Bucle de entrenamiento
- Las llamadas `optimizer.step()`, `loss.backward()`, `optimizer.zero_grad()`.
- Las llamadas `model.eval()`, para deshabilitar los gradientes durante la evaluación.
- Carga y salvado de checkpoints.
- Logueo del experimento (usando tensorboard)
- Entrenamiento en múltiples GPUs y TPUs
- Entrenamiento con precisión de 16 bits

In [ ]:
# Por defecto utilizará todas las gpus disponibles
trainer = pl.Trainer(max_epochs=5, accelerator="auto", devices="auto", strategy="auto")
trainer.fit(model=model, datamodule=mnist)

### Inferencia
Una vez que hemos entrenado nuestro autoencoder vamos a aplicarlo a los datos de test. La idea es imprimir las representaciones en el espacio latente de estas imágenes.

In [ ]:
import torch
latent_space_embeddings = trainer.predict(model, datamodule=mnist)
latent_space_embeddings = torch.concat(latent_space_embeddings, axis=0)

### Mostrando los resultados

En este caso vamos a usar como método de representación un gráfico en 3D para poder visulizar los cluster formados.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib widget

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Visualizar las representaciones en el espacio latente
scatter = ax.scatter(
    latent_space_embeddings[:, 0],
    latent_space_embeddings[:, 1],
    latent_space_embeddings[:, 2],
    c=mnist.mnist_test.targets,
    cmap='tab10'
)

# Añadir los colores
plt.colorbar(scatter)

# Etiquetas para los ejes
ax.set_xlabel('Dimension 1')
ax.set_ylabel('Dimensión 2')
ax.set_zlabel('Dimensión 3')

# Mostrar el plot
plt.show()


### Ejercicios propuestos
1. Prueba a entrenar el autoencoder con el conjunto FashionMNIST. ¿Consigues los mismos resultados?
2. Ejecuta este notebook en Google Colab en un entorno con GPU. Prueba en el Trainer a cambiar el acelerados y observa como de manera transparente puedes cambiar de dispositivo de entrenamiento.
3. Observa el directorio lighning_logs y analiza la inforamción que hay en él.
4. Prueba este mismo ejemplo usando Fashion-MNIST. ¿Qué resultados obtienes?
5. Crea un script para realizar un entrenamiento básico utilizando MNIST para clasificación. Prueba tu modelo entrenado con PyTorch Lightning utilizando los datos de test. ¿Te resulta más o menos fácil que hacerlo con PyTorch directamente?